## 1. 스파크 클러스터 접속

Spark Context 생성시켜 `sc`변수에 지정하고 이를 연결지점으로 스파크 클러스터에 접속한다.

In [1]:
import findspark
findspark.init()
import pyspark

sc = pyspark.SparkContext()

## 2. `sc.readText()` 데이터 불러오기
세익스피어 전집을 구텐베르그 웹사이트에서 텍스트 형태로 다운로드 받는다. [Project Gutenberg’s The Complete Works of William Shakespeare, by William
Shakespeare](http://www.gutenberg.org/files/100/100-0.txt). 이를 `data/` 디렉토리에 저장시킨다.
`shakespeare_complete_work.txt` 외부 `.txt` 데이터를  `sc.textFile()` 함수로 불러와서 RDD 객체로 변환시킨다.
그리고 나서, `flatMap()` 함수를 사용해서 공백을 기준으로 단어별로 토막을 낸다. 

In [8]:
book_rdd = sc.textFile("../data/shakespeare_complete_work.txt")

book_split_rdd = book_rdd.flatMap(lambda x: x.split())

print("토막낸 전체 단어수:", book_split_rdd.count())

토막낸 전체 단어수: 960125


## 3. 세익스피어 전집 데이터 전처리

불용어를 정의해서 이를 제외하고 가장 빈도수가 높은 단어를 10개 추려보는 것을 목표로 삼는다.
1. [NLTK 불용어 사전](https://gist.github.com/sebleier/554280)을 리스트로 정리한다.
1. 토막낸 RDD 단어를 소문자로 변환하고 불용어가 아닌 것만 추려 `book_split_no_stop_rdd` RDD로 작성한다.
1. 튜플로 각 단어와 빈도수 1을 갖는 RDD를 생성한다.
1. `reduceByKey()` 함수로 단어별 빈도수를 계산한다.
1. 키와 값을 바꾼 후에 내림차순으로 정렬시킨다.
1. 가장 빈도수가 높은 단어를 10개 출력시킨다.

In [10]:
# 불용어 사전
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

# 소문자 변환 후 불용어 제거
book_split_no_stop_rdd = book_split_rdd.filter(lambda x: x.lower() not in stop_words)

# 튜플로 각 단어와 빈도수 1을 갖는 RDD를 생성한다.
book_split_no_stop_word_rdd = book_split_no_stop_rdd.map(lambda w: (w, 1))

# reduceByKey() 함수로 단어별 빈도수를 계산한다.
word_count_rdd = book_split_no_stop_word_rdd.reduceByKey(lambda x, y: x + y)

# 키와 값을 바꾼 후에 내림차순으로 정렬시킨다.
word_count_swap_rdd = word_count_rdd.map(lambda x: (x[1], x[0])).sortByKey(ascending=False)

for word in word_count_swap_rdd.take(10):
    print(word)

(4528, 'thou')
(3915, 'thy')
(3247, 'shall')
(2171, 'good')
(2132, 'would')
(1933, 'Enter')
(1887, 'thee')
(1718, 'hath')
(1641, 'like')
(1566, 'make')
